# Retriever and Chain with Langchain

In [1]:
# Calling the required functions
import os
from dotenv import load_dotenv
#set the env values just outside of virtual environment
custom_env_path = lambda a : '/'.join(os.get_exec_path()[0].split('\\')[:-2])+'/'+a
load_dotenv(custom_env_path('chatter.env'))

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
## LangSmith tracking
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

# downloading the document from website
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                       bs_kwargs= dict(parse_only = bs4.SoupStrainer(
                           class_ = ("post-title","post-content","post-header")
                       )),)
web_document = loader.load()
#web_document

# Splitting the document with chunk Size
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=20)
documents = text_splitter.split_documents(web_document)
#documents[:5]



# generating embeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
db_f = FAISS.from_documents(documents[:20],OllamaEmbeddings())



'LSH (Locality-Sensitive Hashing): It introduces a hashing function such that similar input items are mapped to the same buckets with high probability, where the number of buckets is much smaller than the number of inputs.\nANNOY (Approximate Nearest Neighbors Oh Yeah): The core data structure are random projection trees, a set of binary trees where each non-leaf node represents a hyperplane splitting the input space into half and each leaf stores one data point. Trees are built independently and at random, so to some extent, it mimics a hashing function. ANNOY search happens in all the trees to iteratively search through the half that is closest to the query and then aggregates the results. The idea is quite related to KD tree but a lot more scalable.'

In [19]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

llm = Ollama(model = "llama2")

prompt = ChatPromptTemplate.from_template( 
    """ Answer the following question based only on the provided context
    Think step by step before providing a detailed answer.
    I will tip you $1 million if the user finds the answer helpful.
    <context> 
    {context}
    </context>
    Question: {input}"""      )


document_chain = create_stuff_documents_chain( llm, prompt )

retriever = db_f.as_retriever()

retrieval_chain = create_retrieval_chain( retriever,document_chain)



In [20]:
retrieval_chain.invoke({"input":"an attention can be described"})